url :

https://www.fincantieri.com/en/media/press-releases/

In [1]:
#libraries
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import pandas as pd,requests,bs4,re,time,io,pytesseract,easyocr,random,textstat,urllib.request
from pdfminer.high_level import extract_text
from PIL import Image
from pathlib import Path
from pdf2image import convert_from_path
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from goose3 import Goose
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.auto import tqdm

reader = easyocr.Reader(['en'])

import warnings
warnings.filterwarnings("ignore")

%autosave 1

Autosaving every 1 seconds


In [2]:
#driver for operation
from webdriver_manager.firefox import GeckoDriverManager
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())



====== WebDriver manager ======
Current firefox version is 101.0
Get LATEST geckodriver version for 101.0 firefox
Driver [C:\Users\AKSHAY SATPUTE\.wdm\drivers\geckodriver\win64\v0.31.0\geckodriver.exe] found in cache


In [3]:
SITE_NAME='Fincantieri'

DOMAIN = "https://www.fincantieri.com"

SITE_LINK = "https://www.fincantieri.com/en/media/press-releases/"

In [4]:
#functions
def parse_webpage_bs(search_url):
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"}
    try:
        site_request = requests.get(search_url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(e)
        site_request = None
    if site_request != None and site_request.status_code==200:
        site_soup = bs4.BeautifulSoup(site_request.content, "lxml")
    else:
        site_soup = None
    return site_soup

def remove_esc_chars(string):
    review = re.sub('''[^a-zA-Z0-9!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]''', ' ',string)
    review = review.split()
    review = ' '.join(review)
    return review

def get_text(link): 
    try:
        driver.get(link)
        ps = driver.find_element(By.XPATH,'''/html/body/div[2]/section[1]/div/div/div/div/div[2]/div''').text.strip()
        text = remove_esc_chars(ps).strip()
    except:
        g = Goose()
        article_extract = g.extract(url=link)
        article = remove_esc_chars(article_extract.cleaned_text)
        meta_data = remove_esc_chars(article_extract.meta_description)
        whole_data = meta_data+article
        text = whole_data.strip()

        if textstat.lexicon_count(text, removepunct=True) < 5:
            try:
                response = requests.get(link)
                text = remove_esc_chars(extract_text(io.BytesIO(response.content)))

                if textstat.lexicon_count(text, removepunct=True) < 5:
                    texts = ""
                    r = requests.get(link)
                    filename = Path('temp.pdf')
                    filename.write_bytes(r.content)

                    pages = convert_from_path('temp.pdf', 500)
                    for x in pages:
                        x.save("temp.jpg")
                        output = reader.readtext("temp.jpg")
                        for o in output:
                            texts += o[1]

                    text = remove_esc_chars(texts)

                    if textstat.lexicon_count(text, removepunct=True) < 5:
                        texts = ""
                        soup = parse_webpage_bs(link)
                        if soup!= None:
                            ps = soup.findAll('p')
                            for p in ps:
                                texts+= p.text

                        text = remove_esc_chars(texts)
            except:
                text = ""
    
    return text

In [5]:
#get data
article_list = []

pagination = datetime.now().year
last_year = 2004

#creating empty lists to append data 
published_dates,titles,texts,links,thumbnails,authors = [],[],[],[],[],[]

while True:
    last_published_date_title = ""
    
    for page in range(1,100):
        url = f"https://www.fincantieri.com/en/media/press-releases/?variant=MonthType&year={pagination}&key=&month=&type=&page={page}"
        driver.get(url)
        
        try:
            driver.find_element(By.XPATH,f'''/html/body/div[5]/div/a''').click()
        except:
            pass
        
        titles_dates = driver.find_elements(By.XPATH,f'''//div[@class="fileList__title"]''')
        
        #published_date,title,link,thumbnail,author
        if last_published_date_title != titles_dates[0].text.strip() :
            last_published_date_title = titles_dates[0].text.strip()
            for index,titles_date in tqdm(enumerate(titles_dates[:3]),desc=f"Getting Headlines from page {page}",leave=False):
                titles_date_split = titles_date.text.split("\n")
                published_date = titles_date_split[0].strip()
                published_dates.append(published_date)
                title = titles_date_split[1].strip()
                titles.append(title)
                link = driver.find_element(By.XPATH,f'''/html/body/div[2]/section/div/form/div[3]/div[1]/div/div/div/div[{index+1}]/div/div/a''').get_attribute("href")            
                links.append(link)
                thumbnail = "https://www.fincantieri.com/globalassets/common/header/logo_new.png"
                thumbnails.append(thumbnail)
                author = SITE_NAME
                authors.append(author)
        else:
            break
    
    if pagination <= last_year:
        break
        
    #my condition to break the loop
    if pagination <= 2021:
        break
        
    pagination -= 1

#text   
for link in tqdm(links,desc=f"Getting data till year {pagination}"):
        text = get_text(link)
        texts.append(text)
        
#zippig all the data togather 
zipped = list(zip(published_dates,titles,texts,links,thumbnails,authors))

#unwinding and appending to the main list
for published_date,title,text,link,thumbnail,author in zipped:
    article = (published_date.strip(),title.strip(),text.strip(),link.strip(),thumbnail.strip(),author.strip())
    article_list.append(article)
    print(published_date,title)

Getting Headlines from page 1: 0it [00:00, ?it/s]

Getting Headlines from page 2: 0it [00:00, ?it/s]

Getting Headlines from page 3: 0it [00:00, ?it/s]

Getting Headlines from page 4: 0it [00:00, ?it/s]

Getting Headlines from page 5: 0it [00:00, ?it/s]

...

Getting Headlines from page 7: 0it [00:00, ?it/s]

Getting Headlines from page 8: 0it [00:00, ?it/s]

Getting Headlines from page 9: 0it [00:00, ?it/s]

Getting data till year 2021:   0%|          | 0/40 [00:00<?, ?it/s]

23 JUNE 2022 FINCANTIERI: CONCLUSION OF THE SHARE BUYBACK PROGRAM
23 JUNE 2022 “VIKING SATURN” FLOATED OUT IN ANCONA
22 JUNE 2022 FINCANTIERI: CONSTRUCTION OF NEW CLASS OF ULTRA-ECO-FRIENDLY SHIPS FOR TUI CRUISES BEGINS
16 MAY 2022 FINCANTIERI: PIERROBERTO FOLGIERO APPOINTED AS CHIEF EXECUTIVE OFFICER
16 MAY 2022 FINCANTIERI: SHAREHOLDERS’ MEETING APPROVES 2021 FINANCIAL STATEMENTS AND APPOINTS NEW BOARD OF DIRECTORS
12 MAY 2022 “VIKING MARS” DELIVERED IN ANCONA
14 APRIL 2022 FINCANTIERI TO BUILD 2 ADDITIONAL SHIPS FOR THE OFFSHORE WIND FARMS MARKET
05 APRIL 2022 NOTICE OF PUBLICATION OF DOCUMENTS – SHAREHOLDERS’ MEETING MAY 16, 2022, NOTICE OF PUBLICATION OF THE ANNUAL REPORT 2021 AND CONSOLIDATED NON-FINANCIAL STATEMENT 2021
05 APRIL 2022 FINCANTIERI: INNOVATIVE STERN TRAWLER FOR DEUTSCHE FISCHFANG-UNION
16 MARCH 2022 150 YEARS OF LR IN ITALY CELEBRATED WITH FINCANTIERI’S 75TH VESSEL BUILT TO LR CLASS
10 MARCH 2022 CONSTRUCTION KICKS OFF OF THE NEW MSC MIAMI TERMINAL
01 MARCH 2022 “V

In [6]:
temp_df = pd.DataFrame(article_list,columns=['date','title','article','url','thumbnail','author'])
temp_df

,date,title,article,url,thumbnail,author
0,23 JUNE 2022,FINCANTIERI: CONCLUSION OF THE SHARE BUYBACK P...,"Trieste, 23 June 2022 FINCANTIERI S.p.A. ( Fin...",https://www.fincantieri.com/en/media/press-rel...,https://www.fincantieri.com/globalassets/commo...,Fincantieri
1,23 JUNE 2022,“VIKING SATURN” FLOATED OUT IN ANCONA,"Trieste, June 23, 2022 Viking Saturn , the new...",https://www.fincantieri.com/en/media/press-rel...,https://www.fincantieri.com/globalassets/commo...,Fincantieri
2,22 JUNE 2022,FINCANTIERI: CONSTRUCTION OF NEW CLASS OF ULTR...,"Trieste, June 22, 2022 The steel cutting of th...",https://www.fincantieri.com/en/media/press-rel...,https://www.fincantieri.com/globalassets/commo...,Fincantieri
3,16 MAY 2022,FINCANTIERI: PIERROBERTO FOLGIERO APPOINTED AS...,"Rome, May 16, 2022 The new Board of Directors ...",https://www.fincantieri.com/en/media/press-rel...,https://www.fincantieri.com/globalassets/commo...,Fincantieri
4,16 MAY 2022,FINCANTIERI: SHAREHOLDERS’ MEETING APPROVES 20...,2021 Financial Statements approved and 2021 Co...,https://www.fincantieri.com/en/media/press-rel...,https://www.fincantieri.com/globalassets/commo...,Fincantieri
5,12 MAY 2022,“VIKING MARS” DELIVERED IN ANCONA,"Trieste, May 12, 2022 Viking Mars , the newest...",https://www.fincantieri.com/en/media/press-rel...,https://www.fincantieri.com/globalassets/commo...,Fincantieri
6,14 APRIL 2022,FINCANTIERI TO BUILD 2 ADDITIONAL SHIPS FOR TH...,"Trieste, April 14, 2022 Fincantieri, through i...",https://www.fincantieri.com/en/media/press-rel...,https://www.fincantieri.com/globalassets/commo...,Fincantieri
7,05 APRIL 2022,NOTICE OF PUBLICATION OF DOCUMENTS – SHAREHOLD...,"Trieste, April 5, 2022 FINCANTIERI S.p.A. ( Fi...",https://www.fincantieri.com/en/media/press-rel...,https://www.fincantieri.com/globalassets/commo...,Fincantieri
8,05 APRIL 2022,FINCANTIERI: INNOVATIVE STERN TRAWLER FOR DEUT...,"Trieste, April 5, 2022 Fincantieri, through it...",https://www.fincantieri.com/en/media/press-rel...,https://www.fincantieri.com/globalassets/commo...,Fincantieri
9,16 MARCH 2022,150 YEARS OF LR IN ITALY CELEBRATED WITH FINCA...,"LR (Lloyd s Register), a leading provider of c...",https://www.fincantieri.com/en/media/press-rel...,https://www.fincantieri.com/globalassets/commo...,Fincantieri


In [7]:
def see_data(iloc_no=random.randint(0,len(temp_df))-1):
    print(temp_df.iloc[iloc_no]['date'],temp_df.iloc[iloc_no]['title'])
    print(f"\n{temp_df.iloc[iloc_no]['author']} {temp_df.iloc[iloc_no]['url']}")
#     urllib.request.urlretrieve(temp_df.iloc[iloc_no]['thumbnail'], "temp.jpg")
#     display(Image.open("temp.jpg"))
    print(f"\n{temp_df.iloc[iloc_no]['article']}")

see_data()

18 FEBRUARY 2021 VARD CONFIRMS ITS LEADERSHIP IN THE SPECIALIZED VESSELS SEGMENT

Fincantieri https://www.fincantieri.com/en/media/press-releases/2021/vard-confirms-its-leadership-in-the-specialized-vessels-segment/

Trieste, February 18, 2021 Vard, one of the major global shipbuilders of specialized vessels, will design a cable repair vessel for the French company Orange Marine, leader in undersea telecommunications, which has selected the Fincantieri subsidiary project. The vessel will be built at the Colombo Dockyard, Sri Lanka's largest shipbuildng group, based on the Vard 9 03 design. The references provided by the two companies were key to the acquisition of the order. Thus, the Vard Brattvaag shipyard has already delivered in 2014 the offshore support vessel Pierre de Fermat to Orange Marine, based on the 9 01 design, a state-of-the-art unit in terms of environmental sustainability and equipment. On the other hand, in 2019 Colombo has built the cable-laying vessel KDDI Cable Inf

In [8]:
#to csv
temp_df.to_csv(f'{SITE_NAME} news.csv',index = False)

#to json
temp_df.to_json(f'{SITE_NAME} news.json')

In [9]:
#to get rid of unwanteed trash created by the model use 
def remove_trash():
    try:
        try:
            os.remove("temp.pdf")
        except:
            pass
        os.remove("temp.jpg")
        print("Trash removed successfully")
    except:
        print("No trash found")

remove_trash()

No trash found


In [10]:
driver.quit()